In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

main_experiments_path = "../experiments"


basic_method_names = [
    "svr",
    "pls",
    "gradboost",
    "mlp_0.01",
    "mlp_0.001",
    "mlp_0.0001",
    "rbf_full_0.01",
    "rbf_full_0.001",
    "rbf_full_0.0001",
    "rbf_diag_0.01",
    "rbf_diag_0.001",
    "rbf_diag_0.0001",
    "rbf_scal_0.01",
    "rbf_scal_0.001",
    "rbf_scal_0.0001",
    "rbf_univ_0.01",
    "rbf_univ_0.001",
    "rbf_univ_0.0001",
    "ffn_0.1_0.01",
    "ffn_0.1_0.001",
    "ffn_0.1_0.0001",
    "ffn_1_0.01",
    "ffn_1_0.001",
    "ffn_1_0.0001",
    "ffn_10_0.01",
    "ffn_10_0.001",
    "ffn_10_0.0001",
    "uffn_0.1_0.01",
    "uffn_0.1_0.001",
    "uffn_0.1_0.0001",
    "uffn_1_0.01",
    "uffn_1_0.001",
    "uffn_1_0.0001",
    "uffn_10_0.01",
    "uffn_10_0.001",
    "uffn_10_0.0001",
]


basic_condensed_method_names = [
    "svr",
    "pls",
    "gradboost",
    "mlp",
    "rbf_full",
    "rbf_diag",
    "rbf_scal",
    "rbf_univ",
    "ffn",
    "uffn",
]

method_names = []
condensed_method_names = []
for i in range(3):
    method_names += [str(i+1)+"_"+method for method in basic_method_names]
    condensed_method_names += [str(i+1)+"_"+method for method in basic_condensed_method_names]

data_dims = list(range(1,4))
repetition_ids = list(range(8))
learning_rates = ["0.01", "0.001", "0.0001"]

experiments_data = xr.DataArray(np.zeros((len(data_dims),len(basic_condensed_method_names), len(repetition_ids), len(learning_rates))) - 1, coords=[data_dims, basic_condensed_method_names, repetition_ids, learning_rates], dims=['data_dims','method', 'repetition','lr'])

method_settings = os.listdir(main_experiments_path)

# sort by name
method_settings = sorted(method_settings)

#print all folders in the main experiments path
for method_folder in method_settings:
    # iterate over repetitions

    if method_folder.startswith("."):
        continue

    for repetition_folder in os.listdir(main_experiments_path + "/" + method_folder):
        # iterate over files

        if repetition_folder.startswith("."):
            continue

        ## add this repetition to the experiment df as column
        try:
            ## read the npy file
            train_loss = np.load(main_experiments_path + "/" + method_folder + "/" + repetition_folder + "/data/" + "train_loss.npy")
            val_loss = np.load(main_experiments_path + "/" + method_folder + "/" + repetition_folder +  "/data/"  + "val_loss.npy")
            test_loss = np.load(main_experiments_path + "/" + method_folder + "/" + repetition_folder + "/data/" + "test_loss.npy")
        except Exception as e:
            print("Error reading files for dataset: ", method_folder, e)
            continue
        
        ### extract the min val loss and pick the corresponding test loss

        min_val_loss = np.min(val_loss)
        min_val_loss_index = np.argmin(val_loss)

        min_test_loss = test_loss[min_val_loss_index]

        method_folder_id = int(method_folder.split("_")[1])

        method_description = method_names[method_folder_id - 1]
        method_parts = method_description.split("_")

        data_dim = int(method_parts[0])
        method_name = method_parts[1]

        repetition_id = int(repetition_folder.split("_")[1])

        if method_name == "rbf":
            method_name = method_name + "_" + method_parts[2]

        if len(method_parts) == 2:
            experiments_data.loc[data_dim, method_name, repetition_id, :] = min_test_loss
        else:
            lr = method_parts[-1]
            experiments_data.loc[data_dim, method_name, repetition_id, lr] = min_test_loss



## replace all -1 with nan

#test = experiments_data.where(experiments_data == -1)
#print(test)

experiments_data = experiments_data.where(experiments_data != -1)

print(experiments_data.loc[1,"svr",:,:])


experiments_data



<xarray.DataArray (repetition: 8, lr: 3)> Size: 192B
array([[ 2310.53222444,  2310.53222444,  2310.53222444],
       [10907.80513673, 10907.80513673, 10907.80513673],
       [ 5606.95879561,  5606.95879561,  5606.95879561],
       [22360.14822796, 22360.14822796, 22360.14822796],
       [21951.72935318, 21951.72935318, 21951.72935318],
       [25112.42429179, 25112.42429179, 25112.42429179],
       [ 7671.04355497,  7671.04355497,  7671.04355497],
       [ 8113.56069236,  8113.56069236,  8113.56069236]])
Coordinates:
    data_dims   int64 8B 1
    method      <U9 36B 'svr'
  * repetition  (repetition) int64 64B 0 1 2 3 4 5 6 7
  * lr          (lr) <U6 72B '0.01' '0.001' '0.0001'


<xarray.DataArray (data_dims: 3, method: 10, repetition: 8, lr: 3)> Size: 6kB
array([[[[2.31053222e+03, 2.31053222e+03, 2.31053222e+03],
         [1.09078051e+04, 1.09078051e+04, 1.09078051e+04],
         [5.60695880e+03, 5.60695880e+03, 5.60695880e+03],
         [2.23601482e+04, 2.23601482e+04, 2.23601482e+04],
         [2.19517294e+04, 2.19517294e+04, 2.19517294e+04],
         [2.51124243e+04, 2.51124243e+04, 2.51124243e+04],
         [7.67104355e+03, 7.67104355e+03, 7.67104355e+03],
         [8.11356069e+03, 8.11356069e+03, 8.11356069e+03]],

        [[1.15051082e+03, 1.15051082e+03, 1.15051082e+03],
         [7.90916343e+03, 7.90916343e+03, 7.90916343e+03],
         [7.19981379e+03, 7.19981379e+03, 7.19981379e+03],
         [2.22386165e+04, 2.22386165e+04, 2.22386165e+04],
         [1.46792255e+04, 1.46792255e+04, 1.46792255e+04],
         [1.89352832e+04, 1.89352832e+04, 1.89352832e+04],
         [9.95903820e+03, 9.95903820e+03, 9.95903820e+03],
         [9.11171311e+03, 9.11171311e+03, 9.11171311e+03]],

        [[7.62638052e+02, 7.62638052e+02, 7.62638052e+02],
         [9.57763031e+03, 9.57763031e+03, 9.57763031e+03],
...
         [1.29865817e+05, 1.29869393e+05, 1.29919533e+05],
         [1.31033915e+05, 1.30937549e+05, 1.30047814e+05]],

        [[4.45567621e+02, 4.46549360e+02, 4.44019430e+02],
         [3.88595108e+03, 3.86795999e+03, 3.86244128e+03],
         [1.49804711e+04, 1.50830221e+04, 1.51774593e+04],
         [3.25918660e+04, 3.27630822e+04, 3.25947633e+04],
         [6.88819106e+04, 6.82632964e+04, 6.82723047e+04],
         [9.90315903e+04, 1.01199804e+05, 1.00695721e+05],
         [1.31886174e+05, 1.29895232e+05, 1.29938949e+05],
         [1.31836643e+05, 1.30953551e+05, 1.30065390e+05]],

        [[1.64296883e+00, 2.66463192e+00, 6.62496856e+00],
         [7.63995941e+02, 1.10701727e+03, 2.48017418e+03],
         [1.31521016e+04, 1.28712961e+04, 1.35994462e+04],
         [3.21414629e+04, 3.27856182e+04, 3.28082603e+04],
         [6.82630254e+04, 6.81649233e+04, 6.86345239e+04],
         [9.88932173e+04, 9.90037993e+04, 1.00684105e+05],
         [1.29726138e+05, 1.29908811e+05, 1.30517447e+05],
         [1.31881312e+05, 1.30946042e+05, 1.30053576e+05]]]])
Coordinates:
  * data_dims   (data_dims) int64 24B 1 2 3
  * method      (method) <U9 360B 'svr' 'pls' 'gradboost' ... 'ffn' 'uffn'
  * repetition  (repetition) int64 64B 0 1 2 3 4 5 6 7
  * lr          (lr) <U6 72B '0.01' '0.001' '0.0001'

In [6]:
_experiments_data = experiments_data.copy()


repetition_dim_mean_data = _experiments_data.mean(dim=['data_dims', 'repetition'])


min_mlp_mean = repetition_dim_mean_data.sel(dict(method="mlp")).min()
repetition_dim_mean_data = repetition_dim_mean_data / min_mlp_mean
_experiments_data = _experiments_data / min_mlp_mean


repetition_dim_std_data = _experiments_data.std(dim=['data_dims', 'repetition'])

repetition_dim_summary_data = xr.concat([repetition_dim_mean_data, repetition_dim_std_data], dim= pd.Index(["mean", "std"], name='summary'))

means = repetition_dim_summary_data.sel(summary="mean")
min_mean_index = means.argmin(dim="lr")
min_mean_index = min_mean_index.drop_vars('summary')
min_mean_index = min_mean_index.squeeze()

collapsed_repetition_dim_summary_data = repetition_dim_summary_data.isel(lr=min_mean_index, missing_dims="ignore")
collapsed_repetition_dim_summary_data = collapsed_repetition_dim_summary_data.drop_vars('lr')

collapsed_repetition_dim_summary_data

<xarray.DataArray (summary: 2, method: 10)> Size: 160B
array([[0.3144673 , 0.29331077, 0.34133279, 1.        , 0.96971392,
        0.96624217, 0.97133379, 0.95292451, 1.01118681, 0.99621482],
       [0.27356296, 0.22897642, 0.30615838, 1.58069698, 1.58487789,
        1.58448336, 1.58514676, 1.58321762, 1.56272765, 1.56792445]])
Coordinates:
  * method   (method) <U9 360B 'svr' 'pls' 'gradboost' ... 'ffn' 'uffn'
  * summary  (summary) object 16B 'mean' 'std'

In [7]:
_experiments_data = experiments_data.copy()


### average over repetitions and pick the min value across learning rates

### average over repetitions and pick the min value across learning rates
repetition_mean_data = _experiments_data.mean(dim='repetition')
min_mlp_mean = repetition_mean_data.sel(dict(method="mlp")).min(dim='lr')

print("min mlp ",min_mlp_mean.shape, "min mlp", min_mlp_mean)
repetition_mean_data = repetition_mean_data / min_mlp_mean

_experiments_data = _experiments_data / min_mlp_mean 
repetition_std_data = _experiments_data.std(dim='repetition')
## combine mean and std into one xarray

repetition_summary_data = xr.concat([repetition_mean_data, repetition_std_data], dim= pd.Index(["mean", "std"], name='summary'))

### now keep the min mean value across learning rates for each method and data dimension combination and also keep the corresponding std value 
#min_mean_lr = repetition_summary_data.loc[dict(summary='mean')].argmin(dim='lr')

means = repetition_summary_data.sel(summary="mean")
min_mean_index = means.argmin(dim="lr")

print("min mean index: ",min_mean_index)

min_mean_index = min_mean_index.drop_vars('summary')

min_mean_index = min_mean_index.squeeze()

collapsed_repetition_summary_data = repetition_summary_data.isel(lr=min_mean_index, missing_dims="ignore")
collapsed_repetition_summary_data = collapsed_repetition_summary_data.drop_vars('lr')
collapsed_repetition_summary_data

min mlp  (3,) min mlp <xarray.DataArray (data_dims: 3)> Size: 24B
array([ 9206.10632324,  5548.06118822, 59633.24827566])
Coordinates:
  * data_dims  (data_dims) int64 24B 1 2 3
    method     <U9 36B 'mlp'
min mean index:  <xarray.DataArray (data_dims: 3, method: 10)> Size: 240B
array([[0, 0, 0, 1, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 2, 1]])
Coordinates:
  * data_dims  (data_dims) int64 24B 1 2 3
  * method     (method) <U9 360B 'svr' 'pls' 'gradboost' ... 'ffn' 'uffn'
    summary    <U4 16B 'mean'


<xarray.DataArray (summary: 2, data_dims: 3, method: 10)> Size: 480B
array([[[1.41257062, 1.23808266, 1.5480951 , 1.        , 0.85423421,
         0.85423421, 0.85379889, 0.8580138 , 1.00283042, 1.00283042],
        [1.04275269, 1.04799855, 1.17483797, 1.        , 0.96406106,
         0.96406106, 0.98863419, 0.92520981, 1.06757734, 1.01611607],
        [0.07778229, 0.07780041, 0.07813478, 1.        , 0.98990483,
         0.98556754, 0.98933763, 0.97196075, 1.00835327, 0.99522956]],

       [[0.89245889, 0.69468304, 0.97323401, 0.66308688, 0.73732191,
         0.73732191, 0.73754467, 0.68383776, 0.67049247, 0.67049247],
        [0.67418479, 0.68263598, 0.8994719 , 0.74799307, 0.75114559,
         0.75114559, 0.73571418, 0.78689372, 0.68493701, 0.70118096],
        [0.06670254, 0.06640211, 0.06806538, 0.87877369, 0.87911283,
         0.88162367, 0.88036565, 0.88900479, 0.85882658, 0.87003857]]])
Coordinates:
  * data_dims  (data_dims) int64 24B 1 2 3
  * method     (method) <U9 360B 'svr' 'pls' 'gradboost' ... 'ffn' 'uffn'
  * summary    (summary) object 16B 'mean' 'std'

In [8]:
method_renaming = {
    "svr": "SVR",
    "gradboost": "GradBoost",
    "mlp": "MLP",
    "rbf_full": "RBF Full",
    "rbf_diag": "RBF Diag",
    "rbf_scal": "RBF Scal",
    "rbf_univ": "U-RBF",
    "ffn": "FFM",
    "uffn": "U-FFM",
}

### rename the method coords and only keep the methods that have a mapping

collapsed_repetition_summary_data = collapsed_repetition_summary_data.sel(dict(method=[key for key in method_renaming.keys() if key in collapsed_repetition_summary_data.method.values]))#.sel(method=[key for key in method_renaming.keys() if key in method_renaming.keys()])
collapsed_repetition_summary_data.coords['method'] = [method_renaming[method] for method in collapsed_repetition_summary_data.method.values]



mean_summary_data = collapsed_repetition_summary_data.sel(summary="mean")
std_summary_data = collapsed_repetition_summary_data.sel(summary="std") 



mean_df = mean_summary_data.to_dataframe(name='value').reset_index()
mean_df = mean_df.pivot_table(index='data_dims', columns='method', values='value', aggfunc=lambda x: list(x)[0])

std_df = std_summary_data.to_dataframe(name='value').reset_index()
std_df = std_df.pivot_table(index='data_dims', columns='method', values='value', aggfunc=lambda x: list(x)[0])


## rename the row index 
mean_df.index = [str(dim) + " m"  for dim in mean_df.index]

std_df.index = [str(dim) + " s"  for dim in std_df.index]


### merge the mean and std dataframes into one and alternate the rows

final_df = pd.concat([mean_df, std_df], axis=0)
final_df = final_df.sort_index()


collapsed_repetition_dim_summary_data = collapsed_repetition_dim_summary_data.sel(dict(method=[key for key in method_renaming.keys() if key in collapsed_repetition_dim_summary_data.method.values]))#.sel(method=[key for key in method_renaming.keys() if key in method_renaming.keys()])
collapsed_repetition_dim_summary_data.coords['method'] = [method_renaming[method] for method in collapsed_repetition_dim_summary_data.method.values]


mean_datasets_summary_data = collapsed_repetition_dim_summary_data.sel(summary="mean")
std_datasets_summary_data = collapsed_repetition_dim_summary_data.sel(summary="std")

mean_datasets_summary_data = mean_datasets_summary_data.to_dataframe(name='value').reset_index()
mean_datasets_summary_data = mean_datasets_summary_data.pivot_table( columns='method', values='value', aggfunc=lambda x: list(x)[0])

std_datasets_summary_data = std_datasets_summary_data.to_dataframe(name='value').reset_index()
std_datasets_summary_data = std_datasets_summary_data.pivot_table( columns='method', values='value', aggfunc=lambda x: list(x)[0])

## rename the row index
mean_datasets_summary_data.index = [str(dim) + " m"  for dim in mean_datasets_summary_data.index]

std_datasets_summary_data.index = [str(dim) + " s"  for dim in std_datasets_summary_data.index]


final_datasets_summary_df = pd.concat([mean_datasets_summary_data, std_datasets_summary_data], axis=0)
final_datasets_summary_df = final_datasets_summary_df.sort_index()

final_df = pd.concat([final_df, final_datasets_summary_df], axis=0)


final_df


method,FFM,GradBoost,MLP,RBF Diag,RBF Full,RBF Scal,SVR,U-FFM,U-RBF
1 m,1.002830,1.548095,1.000000,0.854234,0.854234,0.853799,1.412571,1.002830,0.858014
1 s,0.670492,0.973234,0.663087,0.737322,0.737322,0.737545,0.892459,0.670492,0.683838
2 m,1.067577,1.174838,1.000000,0.964061,0.964061,0.988634,1.042753,1.016116,0.925210
2 s,0.684937,0.899472,0.747993,0.751146,0.751146,0.735714,0.674185,0.701181,0.786894
3 m,1.008353,0.078135,1.000000,0.985568,0.989905,0.989338,0.077782,0.995230,0.971961
3 s,0.858827,0.068065,0.878774,0.881624,0.879113,0.880366,0.066703,0.870039,0.889005
value m,1.011187,0.341333,1.000000,0.966242,0.969714,0.971334,0.314467,0.996215,0.952925
value s,1.562728,0.306158,1.580697,1.584483,1.584878,1.585147,0.273563,1.567924,1.583218
